In [ ]:
import datetime


class TimeStamp:
    """
    Обернул в класс методы которые возвращают timestamp
    Может вернуть дату + время , а может только время
    """
    def get_time_stamp(self) -> str:
        return datetime.datetime.today().strftime('%H:%M:%S')

    def get_datetime_stamp(self) -> str:
        return datetime.datetime.today().strftime('%c')


class PlcVariable:
    """
    This class contains plc.variable information:
    path, status, value, name.
    """

    def __init__(self,name:str,value:int,plc) -> None:
        self.name = name
        self.value = value
        self.plc = plc
        self.history = [self.value]


    def __str__(self) -> str:
        return f"Name: {self.name}, Value: {self.value}"


    def diff_checking(func):
        """ Декоратор в котором сверяется прошлое значение переменной и нынешнее"""
        def checnking(self):
            prev_value = self.value

            func(self)

            if prev_value != self.value:
                print(f'{TimeStamp().get_datetime_stamp()} - Value of {self.name} has been changed from {prev_value} to {self.value}')
                self.history.append(self.value) # Добавляем новое значение в history, типа отслеживаем изменение
                print(f'History: {self.history}<--Now')
        return checnking


    def get_name(self):
        return self.name


    def get_value(self):
        return self.value

    @diff_checking
    def update_value(self):
        self.value = self.plc.read_by_name(self.name,pyads.PLCTYPE_INT)  


class VariableContainer:
    """
    В контейнер собираем все переменные. Обновляем их и проверяем статус.
    """
    def __init__(self,plc) -> None:
        self.container = []
        self.plc = plc

    def __str__(self) -> str:
        """ Container str representation """
        return f'Container:\n {self.container}'

    def get_names(self)-> list:
        """ Return a list of variables names\paths are contained in the container """
        return [variable.name for variable in self.container]

    def add_to_container(self,plc_variable):
        """ Add new PlcVariable instance into container"""
        self.container.append(plc_variable)

    def show_container(self) -> None:
        """
        Print the contents of the container
        It looks like: "'Name' 100"
        """
        for plc_variable in self.container:
            print(plc_variable.name,plc_variable.value)
    
    def update_container(self):
        """
        ***NOT USED***
        Refresh all values in the container's PLC variables
        Use pyads.Connection.read_list_by_name - method
        But may be it's better to read values from the plc one by one.
        TODO: check it. Compare these two ways.
        """
        if self.container:
            values = self.plc.read_list_by_name(container.get_names())
            self.container = []
            for name in values:
                self.add_to_container(PlcVariable(
                                        name=name,
                                        value=values[name],
                                        plc=self.plc))
            return self.container
    
    def update_variables(self)-> None:
        """ Update all the conteiner's variables one by one with PlcVariable.update_value()"""
        if self.container:
            for item in self.container:
                item.update_value()


class CenterLine:
    def __init__(self,name,setpoint,plc_variable : PlcVariable, maximum : int=None, minimum : int=None) -> None:
        self.name = name
        self.setpoint = setpoint
        self.plc_variable = plc_variable 
        self.status = False

        if not maximum and not minimum:
            self.maximum = self.setpoint
            self.minimum = self.setpoint   # Если не задали пределы, то пределы пусть будут равны значению.
        else:
            self.maximum = maximum
            self.minimum = minimum

        self.update_status()

    def __str__(self)-> str:
        return f"CL-'{self.name}'.Setpoint-{self.setpoint}. PLC Value-{self.get_plc_value()}. Status-{self.get_str_status()}"
    

    def get_plc_value(self)-> int:
        return int(self.plc_variable.value)


    def update_status(self) -> None:
        if self.get_plc_value() == self.setpoint or self.in_range():
            self.status = True
        else:
            self.status = False


    def get_str_status(self) -> str:
        if self.status: return 'OK'
        else: return 'NOK'


    def in_range(self) -> bool:
        return self.minimum <= self.get_plc_value() <= self.maximum        


In [ ]:
position_var = PlcVariable('MAIN/POSITION',100,plc)
cl = CenterLine('Position',100,position_var,minimum=90,maximum=105)
print(cl)
position_var.value =101
cl.update_status()
print(cl)

In [ ]:
def get_var_list(plc)-> list: 
    try:
        symbols=plc.get_all_symbols()
        variables = []

        for s in symbols:
            if s.name[0] != '.':
                variables.append(s.name)

        return variables
    except:
        print('He have an error while we read stuff')


In [ ]:
from time import sleep
import pyads

from configurator import Configurator
from plcvariable import PlcVariable,VariableContainer
import centerline

REMOTE_IP = '192.168.1.177'
REMOTE_ADS = '192.168.1.177.1.1'


def get_vars_from_plc(plc,var_list) -> dict:
    return plc.read_list_by_name(var_list)

def get_configuration(path=None):
    config = Configurator()
    config.read_configuration()
    return config.get_cl_and_plc_paths()

def fill_container(container,dump):
    for v in dump:
        var_container.add_to_container(
            PlcVariable(name=v,value=dump[v],plc=plc)
            )


with pyads.Connection(REMOTE_ADS, pyads.PORT_TC2PLC1, REMOTE_IP) as plc:
    # Read existing configuration
    cl_config,variable_list = get_configuration()

    # Read dump of required variables
    dump = get_vars_from_plc(plc,variable_list)

    # Create CenterLine list with cfg data
    cl_list = centerline.CenterLineList(cl_config)

    # Create container for plc_variable objects
    var_container = VariableContainer(plc)
    
    # Connect PLC_VARS with CL_list instance
    cl_list.attach_from_conteiner(var_container.get_plcvariales())
    
    fill_container(var_container, dump)

    var_container.update_variables()

    # while True:
    #     var_container.update_variables()
    #     sleep(1)
    

In [ ]:
import centerline

draft_cl = {'path':'MAIN/POSITION','setpoint':100,'min':90,'max':105}

print(draft_cl['path'])

position_var = PlcVariable('MAIN/POSITION',100,plc)
cl = CenterLine('Position',100,position_var,minimum=90,maximum=100)
print(cl)
position_var.value =88
cl.update_status()
print(cl)
position_var.value = 99
cl.update_status()
print(cl)

       

In [ ]:
from configurator import Configurator

config = Configurator()
config.read_configuration()

cl,plc = config.get_cl_and_plc_paths()

print(cl)
print()


In [ ]:
import centerline
from configurator import Configurator

config = Configurator()
config.read_configuration()

cl,plc = config.get_cl_and_plc_paths()

cl_list = centerline.CenterLineList(cl)

for cl in cl_list.cl_list:
    print(cl)
print()
for v in plc:
    print(v)

print()
varsplc = var_container.get_plcvariales()
print(varsplc ,'\n')


cl_list.attach_from_conteiner(var_container.get_plcvariales())
for i in cl_list.cl_list:
    print(i)


In [ ]:
def attach_from_conteiner(cl_list,plcvar_list):
    for index,item in enumerate(cl_list.cl_list):
        #print(f'Trying to find {item.path}')
        for plc_var in plcvar_list:
            if item.path == plc_var.name:
                #print(f'We found {item.path} in {plc_var.name}')
                item.set_plc_variable(plc_var)
            else:
                pass
                #print("\tChange nothing\n")

attach_from_conteiner(cl_list,var_container.get_plcvariales())
for i in cl_list.cl_list:
    print(i)

In [ ]:
import random

base = ['one','two','three', 'four']
data = ['one','privet','two','three', 'four','five',123]
random.shuffle(data)


print(base)
print(data)
#a = [base[base.index(item)].strip('o') for item in base if item in data]
print(base)
for i,item in enumerate(base):
    print(f'Trying to find {item}')
    for j in data:
        if item == j:
            print(f'\tFind {item}! Change {base[i]} to {base[i]}+!')
            base[i] = base[i]+'!'
        else:
            print("\tChange nothing")

print(base)
print(data)


In [10]:
import datetime

class TaskTimer:
    """ Класс для реализации таймера вызова Task """
    def __init__(self,update_period:int,task)->None:
        self.update_period = update_period
        self.task = task
        self.start_time = 0

    def execute_task(self):
        self.task()

    def start_count(self):
        self.start_time = datetime.datetime.now()

    def check(self):
        pass

    def run(self):
        pass

In [11]:
def anotherTask():
    print('Running!')

timer = TaskTimer(1,anotherTask)

In [8]:
import datetime

tick = False
cycle = 0
cycles = []
period = datetime.timedelta(seconds=3)
start_time = datetime.datetime.now()

while cycle != 10:
    if datetime.datetime.now() >= start_time + period:
        print('Tick',cycle)
        cycles.append(cycle)
        tick = True
        start_time = datetime.datetime.now()
        cycle += 1





Tick 0
Tick 1
Tick 2
Tick 3
Tick 4
Tick 5
Tick 6
Tick 7
Tick 8
Tick 9
